In [1]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np

from sklearn.metrics import mean_squared_error
from math import sqrt

Using TensorFlow backend.


In [2]:
input_img = Input(shape=(784,))
encoded = Dense(32, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(inputs=input_img, outputs=decoded)

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [3]:
(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [4]:
print(x_train.shape)
print(x_test.shape)


(60000, 784)
(10000, 784)


In [5]:
autoencoder.fit(x_train, x_train,
                epochs=200,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))



Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 5s 81us/step - loss: 0.3625 - val_loss: 0.2709
Epoch 2/200
60000/60000 [==============================] - 4s 69us/step - loss: 0.2634 - val_loss: 0.2516
Epoch 3/200
60000/60000 [==============================] - 6s 92us/step - loss: 0.2410 - val_loss: 0.2281
Epoch 4/200
60000/60000 [==============================] - 5s 88us/step - loss: 0.2210 - val_loss: 0.2114
Epoch 5/200
60000/60000 [==============================] - 5s 87us/step - loss: 0.2069 - val_loss: 0.1996
Epoch 6/200
60000/60000 [==============================] - 5s 85us/step - loss: 0.1965 - val_loss: 0.1906
Epoch 7/200
60000/60000 [==============================] - 5s 80us/step - loss: 0.1881 - val_loss: 0.1830
Epoch 8/200
60000/60000 [==============================] - 5s 82us/step - loss: 0.1809 - val_loss: 0.1763
Epoch 9/200
60000/60000 [==============================] - 5s 92us/step - loss: 0.1748 - val_loss: 0.1

60000/60000 [==============================] - 4s 64us/step - loss: 0.0986 - val_loss: 0.0971
Epoch 78/200
60000/60000 [==============================] - 4s 60us/step - loss: 0.0985 - val_loss: 0.0970
Epoch 79/200
60000/60000 [==============================] - 4s 64us/step - loss: 0.0984 - val_loss: 0.0969
Epoch 80/200
60000/60000 [==============================] - 4s 65us/step - loss: 0.0983 - val_loss: 0.0968
Epoch 81/200
60000/60000 [==============================] - 4s 65us/step - loss: 0.0982 - val_loss: 0.0967
Epoch 82/200
60000/60000 [==============================] - 4s 69us/step - loss: 0.0981 - val_loss: 0.0966
Epoch 83/200
60000/60000 [==============================] - 4s 62us/step - loss: 0.0980 - val_loss: 0.0965
Epoch 84/200
60000/60000 [==============================] - 4s 68us/step - loss: 0.0979 - val_loss: 0.0965
Epoch 85/200
60000/60000 [==============================] - 5s 78us/step - loss: 0.0979 - val_loss: 0.0964
Epoch 86/200
60000/60000 [========================

60000/60000 [==============================] - 4s 73us/step - loss: 0.0957 - val_loss: 0.0944
Epoch 154/200
60000/60000 [==============================] - 4s 63us/step - loss: 0.0957 - val_loss: 0.0944
Epoch 155/200
60000/60000 [==============================] - 4s 64us/step - loss: 0.0957 - val_loss: 0.0943
Epoch 156/200
60000/60000 [==============================] - 5s 81us/step - loss: 0.0956 - val_loss: 0.0943
Epoch 157/200
60000/60000 [==============================] - 5s 81us/step - loss: 0.0956 - val_loss: 0.0943
Epoch 158/200
60000/60000 [==============================] - 4s 72us/step - loss: 0.0956 - val_loss: 0.0943
Epoch 159/200
60000/60000 [==============================] - 4s 70us/step - loss: 0.0956 - val_loss: 0.0943
Epoch 160/200
60000/60000 [==============================] - 4s 73us/step - loss: 0.0956 - val_loss: 0.0943
Epoch 161/200
60000/60000 [==============================] - 4s 70us/step - loss: 0.0956 - val_loss: 0.0942
Epoch 162/200
60000/60000 [===============

In [6]:
nbTest=10
patchsize=28

Testimage=np.zeros((nbTest,patchsize*patchsize));
GroundTruth=np.zeros((nbTest,patchsize*patchsize))
OBSvec=np.zeros((nbTest,patchsize*patchsize))
for i in range(nbTest):             
    Testimage[i]=x_test[i,:].copy()
    GroundTruth[i]=x_test[i,:].copy()
    
    missRate=0.5
    missInd=np.nonzero(np.random.choice([0, 1], size=(Testimage.shape[1]), p=[1-missRate, missRate]))
    Testimage[i,missInd[0]]=float('nan')
    OBSvec[i]=Testimage[i].copy()


In [7]:
ImageReconstructedResult=np.zeros((nbTest,patchsize*patchsize));
rmseFinal=[]

for ii in range(nbTest): 
    rmseVec=[];
    indMissingTest=np.where(np.isnan(Testimage[ii]))[0]
    Testimage[ii,indMissingTest]=0 #np.nanmean(Testimage); #or simply 0
    # iter 1
    iteration=1
    tempmax=np.amax(Testimage[ii])
    Testimage[ii]=Testimage[ii]/tempmax
    ImageReconstructed = autoencoder.predict(Testimage[None,ii])
    Testimage[ii,indMissingTest]=ImageReconstructed[0,indMissingTest]
    
    
    rmseError1 = sqrt(mean_squared_error(GroundTruth[ii],ImageReconstructed[0]*tempmax))
    rmseVec.append(rmseError1)
    print(rmseVec)
    # next iterations
    rmseError=rmseError1
    OldrmseError=rmseError
    NewrmseError=0
    
    
    for j in range(1,50):
        OldrmseError=rmseError
        ImageReconstructed = autoencoder.predict(Testimage[None,ii]) 
        Testimage[ii,indMissingTest]=ImageReconstructed[0,indMissingTest];
        rmseError = sqrt(mean_squared_error(GroundTruth[ii],ImageReconstructed[0]*tempmax))
        iteration=iteration+1
        NewrmseError=rmseError
        rmseVec.append(rmseError)
        ImageReconstructedResult[ii,:]=ImageReconstructed[0].copy()
        
        if NewrmseError < OldrmseError:
            ImageReconstructedResult[ii,:]=ImageReconstructed[0].copy();
            #print(j)
            continue
        else:
            break
        
    #print(rmseVec)
    rmseFinal.append(rmseVec[-1])
    
#print("rmsefinal is %f" %np.mean(rmseFinal))


[0.1785827919351952]
[0.2546623718318319]
[0.16786465399182715]
[0.2892883259950713]
[0.2112757737191213]
[0.17591778060464075]
[0.24525503909498478]
[0.2275084678248235]
[0.24532249515903848]
[0.27619300014693016]


In [8]:
################" reconstruction

import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(GroundTruth[i].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display corrupted
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(OBSvec[i].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(3, n, i + 1 + 2*n)
    plt.imshow(ImageReconstructedResult[i].reshape(28, 28))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)


